In [1]:
import numpy as np
import pandas as pd

In [97]:
# 변수명, 측정값, 최솟값, 최댓값, 망대 여부, 퍼지 여부
eval_data = [['공급의 신속성', 3, 0, 5, True, False],
             ['검사불합격', 3.4, 0, 6, False, False],
             ['품질 만족도', (4.5, 6), (0, 1), (6, 7), True, True],
             ['불편함', (1.5, 2), (0, 1), (9, 10), False, True]
            ]

In [98]:
# 입력 데이터
pd.DataFrame(eval_data, columns=['항목', '측정값', '최솟값', '최댓값', '망대 여부', 'Fuzzy 여부'])

,항목,측정값,최솟값,최댓값,망대 여부,Fuzzy 여부
0,공급의 신속성,3,0,5,True,False
1,검사불합격,3.4,0,6,False,False
2,품질 만족도,"(4.5, 6)","(0, 1)","(6, 7)",True,True
3,불편함,"(1.5, 2)","(0, 1)","(9, 10)",False,True


In [99]:
# 각 항목의 fuzzy 만족도 지수를 계산하는 함수
def get_fuzzy_score(data):
    y_i = data[1]                             # 측정값
    y_min = data[2]; y_max = data[3]          # 최솟값; 최댓값
    isMangdae = data[4]; isFuzzy = data[5]    # 망대 여부; Fuzzy 여부
    
    if isMangdae:    # 망대 특성인 경우
        if isFuzzy:  # Fuzzy 데이터인 경우
            # elementwise 연산를 위해 nunmpy array로 변환
            y_min = np.array(y_min)
            y_max = np.array(y_max)
            y_i = np.array(y_i)
            
            ################# interval 간의 크기 비교는 어떻게???? (일단 중점끼리 비교) 
            if np.mean(y_i) < np.mean(y_min):
                score = (0, 0)
            elif np.mean(y_i) > np.mean(y_max):
                score = (1, 1)
            else:
                bunja = y_i - y_min
                bunmo = y_max - y_min
                cases = [a/b for a in bunja for b in bunmo]       # 분자/분모의 구간 연산
                score = (min(cases), max(cases))
                
            return score
        
        else:       # Crisp 데이터인 경우
            if y_i <= y_min:
                score = 0
            elif y_i >= y_max:
                score = 1
            else:
                score = (y_i - y_min) / (y_max - y_min)
            
            return (score, score)
     
    
    else:    # 망소 특성인 경우 -> 1에서 뺀 값을 반환
        if isFuzzy:  # Fuzzy 데이터인 경우
            # elementwise 연상를 위해 nunmpy array로 변환
            y_min = np.array(y_min)
            y_max = np.array(y_max)
            y_i = np.array(y_i)
            
            ################# interval 간의 크기 비교는 어떻게???? (일단 중점끼리 비교) 
            if np.mean(y_i) < np.mean(y_min):
                score = (1, 1)
            elif np.mean(y_i) > np.mean(y_max):
                score = (0, 0)
            else:
                bunja = y_i - y_min
                bunmo = y_max - y_min
                cases = [a/b for a in bunja for b in bunmo]        # 분자/분모의 구간 연산
                score = (1-max(cases), 1-min(cases))
            
            return score
        
        else:       # Crisp 데이터인 경우
            if y_i <= y_min:
                score = 0
            elif y_i >= y_max:
                score = 1
            else:
                score = (y_i - y_min) / (y_max - y_min)
            
            return (1-score, 1-score)

In [119]:
# 하나씩 확인
for data in eval_data:
    score = get_fuzzy_score(data)
    print(f'항목 : {data[0]},\t 만족도 지수 : ({score[0]:.3f}, {score[1]:.3f})')

항목 : 공급의 신속성,	 만족도 지수 : (0.600, 0.600)
항목 : 검사불합격,	 만족도 지수 : (0.433, 0.433)
항목 : 품질 만족도,	 만족도 지수 : (0.750, 0.833)
항목 : 불편함,	 만족도 지수 : (0.833, 0.889)


In [91]:
# 종합 만족도 지수를 계산하는 함수
def get_total(eval_data):
    scores = [get_fuzzy_score(data) for data in eval_data]
    min_ = np.mean([m[0] for m in scores])
    max_ = np.mean([m[1] for m in scores])
    return (min_, max_)

In [109]:
get_total(eval_data)

(0.6541666666666667, 0.6888888888888889)